<a href="https://colab.research.google.com/github/Deinbi/Telecom-X-Parte-2-Predicci-n-de-Cancelaci-n-Churn-/blob/main/Telecom_X_Parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telecom_X_Parte 2

## Paquetes Usados

In [73]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import LabelEncoder


## Extracción del Archivo Tratado

Importo directamente el CSV de github

In [74]:


url = "https://raw.githubusercontent.com/LeoSR02/Challenge3-data-science_TelecomX_parte2/refs/heads/main/datos_tratados.csv"
df = pd.read_csv(url)
df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total,Cuentas_Diarias,Churn_Label
0,0002-ORFBO,False,Female,0,True,True,9,True,False,DSL,...,True,True,False,1 año,True,Cheque por correo,65.6,593.30,2.19,No
1,0003-MKNFE,False,Male,0,False,False,9,True,True,DSL,...,False,False,True,Mensual,False,Cheque por correo,59.9,542.40,2.00,No
2,0004-TLHLJ,True,Male,0,False,False,4,True,False,Fiber optic,...,False,False,False,Mensual,True,Cheque electrónico,73.9,280.85,2.46,Sí
3,0011-IGKFF,True,Male,1,True,False,13,True,False,Fiber optic,...,False,True,True,Mensual,True,Cheque electrónico,98.0,1237.85,3.27,Sí
4,0013-EXCHZ,True,Female,1,True,False,3,True,False,Fiber optic,...,True,True,False,Mensual,True,Cheque por correo,83.9,267.40,2.80,Sí


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7043 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   bool   
 5   Dependents        7267 non-null   bool   
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   bool   
 8   MultipleLines     7267 non-null   bool   
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   bool   
 11  OnlineBackup      7267 non-null   bool   
 12  DeviceProtection  7267 non-null   bool   
 13  TechSupport       7267 non-null   bool   
 14  StreamingTV       7267 non-null   bool   
 15  StreamingMovies   7267 non-null   bool   
 16  Contract          7267 non-null   object 


## Eliminación de Columnas innecesarias

Se eliminan las columnas que no serán necesarias para generar los modelos predictivos.

In [76]:
df.columns

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges_Monthly', 'Charges_Total',
       'Cuentas_Diarias', 'Churn_Label'],
      dtype='object')

In [77]:
df.drop('customerID', axis=1, inplace=True)
df.drop('gender', axis=1, inplace=True)
df.drop('Partner', axis=1, inplace=True)
df.drop('PhoneService', axis=1, inplace=True)
df.drop('MultipleLines', axis=1, inplace=True)
df.drop('InternetService', axis=1, inplace=True)
df.drop('OnlineSecurity', axis=1, inplace=True)
df.drop('OnlineBackup', axis=1, inplace=True)
df.drop('DeviceProtection', axis=1, inplace=True)
df.drop('TechSupport', axis=1, inplace=True)
df.drop('StreamingTV', axis=1, inplace=True)
df.drop('StreamingMovies', axis=1, inplace=True)
df.drop('PaperlessBilling', axis=1, inplace=True)
df.drop('Cuentas_Diarias', axis=1, inplace=True)
df.head(5)

,Churn,SeniorCitizen,Dependents,tenure,Contract,PaymentMethod,Charges_Monthly,Charges_Total,Churn_Label
0,False,0,True,9,1 año,Cheque por correo,65.6,593.30,No
1,False,0,False,9,Mensual,Cheque por correo,59.9,542.40,No
2,True,0,False,4,Mensual,Cheque electrónico,73.9,280.85,Sí
3,True,1,False,13,Mensual,Cheque electrónico,98.0,1237.85,Sí
4,True,1,False,3,Mensual,Cheque por correo,83.9,267.40,Sí


In [78]:
# Cambiar tipo a columna

df['Dependents'] = df['Dependents'].eq('True').astype('Int8')   # Male→1, Female→0
df['Churn']  = df['Churn'].eq('True').astype('Int8')    # True→1, False→0


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Churn            7267 non-null   Int8   
 1   SeniorCitizen    7267 non-null   int64  
 2   Dependents       7267 non-null   Int8   
 3   tenure           7267 non-null   int64  
 4   Contract         7267 non-null   object 
 5   PaymentMethod    7267 non-null   object 
 6   Charges_Monthly  7267 non-null   float64
 7   Charges_Total    7256 non-null   float64
 8   Churn_Label      7043 non-null   object 
dtypes: Int8(2), float64(2), int64(2), object(3)
memory usage: 425.9+ KB


In [80]:
xIn=df.drop(columns='Churn')
yOut=df['Churn']

## Verificación de la Proporción de la Cancelación (Churn)

Calcula la proporción de clientes que cancelaron en relación con los que permanecieron activos. Evalúa si existe un desbalance entre las clases, ya que esto puede impactar en los modelos predictivos y en el análisis de los resultados.

In [81]:
counts = (df['Churn']
          .value_counts(dropna=True)
          .rename_axis('Churn')
          .reset_index(name='n'))
counts['pct'] = counts['n'] / counts['n'].sum()

# Etiqueta categórica para usar en 'color'
counts['Churn_lbl'] = counts['Churn'].map({0:'No', 1:'Sí'}).astype('category')

import plotly.express as px
fig = px.bar(
    counts, x='Churn', y='n', color='Churn_lbl',  # <- usa la etiqueta categórica
    text=counts.apply(lambda r: f"{r['n']:,.0f}<br>({r['pct']:.1%})", axis=1),
    color_discrete_map={'No':'#ff6961', 'Sí':'#59adf6'}
)
fig.update_traces(textposition='outside', cliponaxis=False)
fig.update_xaxes(categoryorder='array', categoryarray=[0, 1])  # orden 0 luego 1
fig.update_layout(
    yaxis_title='Conteo',
    yaxis_range=[0, counts['n'].max()*1.15],
    legend_title_text='Churn',
    title='Distribución de Cancelación (Churn)'
)
fig.show()


In [82]:
df['Churn'].value_counts(normalize=True).mul(100).round(1).map('{:.1f}%'.format)



,proportion
Churn,
0,100.0%


## Encoding
Transforma las variables categóricas a formato numérico para hacerlas compatibles con los algoritmos de machine learning. Utiliza un método de codificación adecuado, como one-hot encoding.

In [83]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Churn            7267 non-null   Int8   
 1   SeniorCitizen    7267 non-null   int64  
 2   Dependents       7267 non-null   Int8   
 3   tenure           7267 non-null   int64  
 4   Contract         7267 non-null   object 
 5   PaymentMethod    7267 non-null   object 
 6   Charges_Monthly  7267 non-null   float64
 7   Charges_Total    7256 non-null   float64
 8   Churn_Label      7043 non-null   object 
dtypes: Int8(2), float64(2), int64(2), object(3)
memory usage: 425.9+ KB


In [84]:
# Variables Categoricas

categoricas = df.select_dtypes(include=['object']).columns.tolist()
print("Variables categóricas:", categoricas)

Variables categóricas: ['Contract', 'PaymentMethod', 'Churn_Label']


In [85]:
xIn.columns

Index(['SeniorCitizen', 'Dependents', 'tenure', 'Contract', 'PaymentMethod',
       'Charges_Monthly', 'Charges_Total', 'Churn_Label'],
      dtype='object')

In [86]:
xIn.head()

,SeniorCitizen,Dependents,tenure,Contract,PaymentMethod,Charges_Monthly,Charges_Total,Churn_Label
0,0,0,9,1 año,Cheque por correo,65.6,593.30,No
1,0,0,9,Mensual,Cheque por correo,59.9,542.40,No
2,0,0,4,Mensual,Cheque electrónico,73.9,280.85,Sí
3,1,0,13,Mensual,Cheque electrónico,98.0,1237.85,Sí
4,1,0,3,Mensual,Cheque por correo,83.9,267.40,Sí


In [87]:
yOut.head()

,Churn
0,0
1,0
2,0
3,0
4,0


## Balanceo de Clases

Si deseas profundizar en el análisis, aplica técnicas de balanceo como undersampling o oversampling. En situaciones de fuerte desbalanceo, herramientas como SMOTE pueden ser útiles para generar ejemplos sintéticos de la clase minoritaria.

In [88]:
df.Churn.value_counts()

,count
Churn,
0,7267
